In [1]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import asyncio
from typing import Optional, Dict, Any, List
from mcp.client.session import ClientSession
from mcp.client.sse import sse_client
from contextlib import asynccontextmanager

class SimpleMCPClient:
    def __init__(self):
        self.session: Optional[ClientSession] = None
        self.tools = None
    
    @asynccontextmanager
    async def connect(self, server_url: str = "http://localhost:8000/sse"):
        """创建SSE连接的上下文管理器"""
        try:
            async with sse_client(server_url) as streams:
                read_stream, write_stream = streams
                async with ClientSession(read_stream, write_stream) as session:
                    self.session = session
                    await self.session.initialize()
                    result = await self.session.list_tools()
                    self.tools = result.tools
                    yield self
        finally:
            self.session = None
            self.tools = None

    async def call_tool(self, tool_name: str, arguments: Dict[str, Any]) -> Any:
        """调用指定的工具"""
        if not self.session:
            raise RuntimeError("Not connected to server")
        
        tool = next((t for t in self.tools if t.name == tool_name), None)
        if not tool:
            raise ValueError(f"Tool {tool_name} not found")
        
        return await self.session.call_tool(tool, arguments)
    
# 在notebook中使用的辅助函数
async def run_in_notebook(coroutine):
    """在notebook中运行异步代码的辅助函数"""
    return await coroutine

In [10]:
# 创建客户端实例
client = SimpleMCPClient()

# 方式1：使用异步上下文管理器
async with client.connect() as c:
    # 列出可用工具
    print("Available tools:", [t.name for t in c.tools])
    print(c.tools)
    # 调用工具示例
    # result = await c.call_tool("tool_name", {"arg": "value"})
    # print("Result:", result)

# # 方式2：使用辅助函数在notebook中运行
# async def main():
#     async with client.connect() as c:
#         tools = c.tools
#         print("Available tools:", [t.name for t in tools])
        
#         # # 调用工具示例
#         # result = await c.call_tool("tool_name", {"arg": "value"})
#         # return result

# # 在notebook中运行
# result = await run_in_notebook(main())

Available tools: ['create-knowledge-folder', 'update-knowledge-folder', 'move-knowledge-folder', 'delete-knowledge-folder', 'get-knowledge-directory', 'get-knowledge-capacity', 'get-knowledge-file-list', 'get-knowledge-file-tags', 'add-knowledge-file-tag', 'remove-knowledge-file-tag', 'get-knowledge-note', 'save-knowledge-note', 'get-scholar-info', 'get-scholar-coauthors', 'search-scholars', 'batch-get-scholars', 'get-scholar-papers', 'get-follow-list', 'get-subscription-list', 'search-papers-normal', 'search-papers-enhanced', 'search-papers-pro-v1', 'search-papers-pro-v2']
[Tool(name='create-knowledge-folder', description='在知识库中创建新文件夹', inputSchema={'type': 'object', 'properties': {'parent_id': {'type': 'integer', 'description': '父文件夹ID'}, 'folder_name': {'type': 'string', 'description': '文件夹名称'}}, 'required': ['parent_id', 'folder_name']}, annotations=None), Tool(name='update-knowledge-folder', description='更新知识库中的文件夹名称', inputSchema={'type': 'object', 'properties': {'folder_id': {'t